In [1]:
from deepface import DeepFace
import os
import matplotlib.pyplot as plt
from retinaface import RetinaFace
import matplotlib.image as mpimg
import time
import numpy as np
import json
import requests
import pandas as pd
import pickle

In [2]:
path = "./face"
images = os.listdir(path)
len(images)

300

In [ ]:
backends = [
  'opencv', 
  'ssd',
  'mtcnn', 
  'fastmtcnn',
  # 'retinaface',
  'yolov8',
  'yunet',
  'centerface',
]
detect_times = {key: 0 for key in backends}
faces_detected = {backend: {} for backend in backends}
for backend in backends:
    for image in images:
        faces_detected[backend][image] = 0 

In [ ]:
for backend in backends:
    print("model: ", backend)
    t = 0
    t1 = time.time()
    for image in images:
        try:
            faces = DeepFace.detectFace(
                img_path = f"{path}/{image}",
                detector_backend = backend,
                enforce_detection = True
            )
            faces_detected[backend][image] = 1
        except:
            faces_detected[backend][image] = 0
    t2 = time.time()
    t += t2-t1
    detect_times[backend] = t
    print("time: ", t)

In [18]:
# face++
import requests
http_url = 'https://api-us.faceplusplus.com/facepp/v3/detect'
# key = '7SNt9yPlw_ITkwUisnXUU43-LTvHwUkD'
# secret = 'IdrN36XnjHe88AdFHio6eyWSn7THc-Gb'

key = "Y9M5bmrjRSmdTprNFbktFnebOy46SmKr"
secret = "hGSqh1y_GRx6cgjRxwn6Tr4lo0aNJxLF" # free trail in USA  

def facepp_detection(filepath):
    boundary = '----------%s' % hex(int(time.time() * 1000))
    data = []
    data.append('--%s' % boundary)
    data.append('Content-Disposition: form-data; name="%s"\r\n' % 'api_key')
    data.append(key)
    data.append('--%s' % boundary)
    data.append('Content-Disposition: form-data; name="%s"\r\n' % 'api_secret')
    data.append(secret)
    data.append('--%s' % boundary)
    fr = open(filepath, 'rb')
    data.append('Content-Disposition: form-data; name="%s"; filename=" "' % 'image_file')
    data.append('Content-Type: %s\r\n' % 'application/octet-stream')
    data.append(fr.read())
    fr.close()
    data.append('--%s' % boundary)
    data.append('Content-Disposition: form-data; name="%s"\r\n' % 'return_landmark')
    data.append('1')
    data.append('--%s' % boundary)
    data.append('Content-Disposition: form-data; name="%s"\r\n' % 'return_attributes')
    data.append(
        "gender,age,smiling,headpose,facequality,blur,eyestatus,emotion,ethnicity,beauty,mouthstatus,eyegaze,skinstatus")
    data.append('--%s--\r\n' % boundary)
    
    for i, d in enumerate(data):
        if isinstance(d, str):
            data[i] = d.encode('utf-8')
    
    http_body = b'\r\n'.join(data)
    
    headers = {
        'Content-Type': 'multipart/form-data; boundary=%s' % boundary
    }
    
    try:
        response = requests.post(http_url, data=http_body, headers=headers, verify=True, timeout=5)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx and 5xx)

        qrcont = response.content
        faces = json.loads(qrcont.decode('utf-8'))['faces']
        print(filepath,len(faces))
        if len(faces) == 0:
            return 0
        else:
            return 1
    except requests.exceptions.RequestException as e:
        # Print or log the error for debugging
        print(f"Request failed: {e}")
        return 0

In [ ]:
faces_detected['facepp'] = {}
for image in images:
    faces_detected['facepp'][image] = 0 

t1 = time.time()
for image in images:
    img_path = f"{path}/{image}"
    if facepp_detection(img_path) == 1:
        faces_detected['facepp'][image] = 1
    else:
        faces_detected['facepp'][image] = 0
    time.sleep(0.34)
t2 = time.time()
t = t2-t1
detect_times['facepp'] = t


# API_KEY1 = "Y9M5bmrjRSmdTprNFbktFnebOy46SmKr"
# API_SECRET1 = "hGSqh1y_GRx6cgjRxwn6Tr4lo0aNJxLF" # free trail in USA  


# API_KEY2 = "7upsWb6lZlawu2nZIwwTAH1Ssg043bQw"
# API_SECRET2 = "zBs40KyB45m_Soig8R-b-5fKqWDexUWy" # free trail in USA

In [ ]:
import time, json, ast, sys, os
import requests
import base64

faces_detected['facepp_modified'] = {}

# API_KEY1 = "7SNt9yPlw_ITkwUisnXUU43-LTvHwUkD"
# API_SECRET1 = "IdrN36XnjHe88AdFHio6eyWSn7THc-Gb" # free API one

# API_KEY2 = "pxgpVU6N4J0RB4-0qcWuSW5AfHFSLaw1"
# API_SECRET2 = "FT1BTtidH6TPaws3LkrE-YWI0OHVfRWe" # free API two


API_KEY1 = "Y9M5bmrjRSmdTprNFbktFnebOy46SmKr"
API_SECRET1 = "hGSqh1y_GRx6cgjRxwn6Tr4lo0aNJxLF" # free trail in USA  


API_KEY2 = "7upsWb6lZlawu2nZIwwTAH1Ssg043bQw"
API_SECRET2 = "zBs40KyB45m_Soig8R-b-5fKqWDexUWy" # free trail in USA

http_url = 'https://api-us.faceplusplus.com/facepp/v3/detect'


nrequests = 0
t1 = time.time()

for img in images:
    imgpath = f"{path}/{img}"

    if 1 == 1:
        image_file = open(imgpath, "rb")
        encoded_string = base64.b64encode(image_file.read())
    
        # resultpath = os.path.join(cwd, "face result",'faceplusplus raw', imgname + ".txt")
        # error_resultpath = os.path.join(cwd, "face result",'faceplusplus raw', imgname + "_error.txt")

        attributes = "gender,age,smiling,headpose,facequality,blur,eyestatus,emotion,ethnicity,beauty,mouthstatus,eyegaze,skinstatus"
        attempts = 0
        while attempts < 5:
            try:
                if nrequests % 2 == 0:
                    API_KEY = API_KEY1; API_SECRET = API_SECRET1
                else:
                    API_KEY = API_KEY2; API_SECRET = API_SECRET2
                # print(nrequests, API_KEY, API_SECRET)


                r = requests.post(http_url,
                             data = {'api_key': API_KEY,
                                     'api_secret': API_SECRET,
                                     'image_base64': encoded_string,
                                     'return_landmark': 1,
                                     'return_attributes': attributes},
                             timeout = 10)
                
                nrequests = nrequests + 1
                qrcont = r.content
                faces = json.loads(qrcont.decode('utf-8'))['faces']
                print(img,len(faces))
                if len(faces) == 0:
                    faces_detected['facepp_modified'][img] = 0
                else:
                    faces_detected['facepp_modified'][img] = 1
                break
            except Exception as e:
                print(e)
                faces_detected['facepp_modified'][img] = 0
                attempts = attempts + 1
                time.sleep(0.1 + attempts * 0.2)
                if attempts > 3: time.sleep(1)
        # if attempts < 5:
        #     wlist = [imgname, nface, rtext]
        #     print(wlist)
        # else:
        #     wlist = [imgname]
        #     print("error")
        #     print(wlist)

t2 = time.time()
detect_times['facepp_modified'] = t2-t1

In [22]:
# time for each backend model
detect_times

{'opencv': 53.13692665100098,
 'ssd': 26.392200469970703,
 'mtcnn': 574.037052154541,
 'fastmtcnn': 143.18174481391907,
 'yolov8': 81.95350646972656,
 'yunet': 21.97808027267456,
 'centerface': 283.7023148536682,
 'facepp': 216.4133312702179,
 'facepp_modified': 214.45973658561707}

In [23]:
backends = [
  'opencv', 
  'ssd',
  'mtcnn', 
  'fastmtcnn',
  # 'retinaface',
  'yolov8',
  'yunet',
  'centerface',
  'facepp',
  'facepp_modified',
]
confusion_matrices = {backend: {} for backend in backends}
for backend in backends:
    confusion_matrices[backend]['tp'] = 0
    confusion_matrices[backend]['fn'] = 0 
    confusion_matrices[backend]['tn'] = 0 
    confusion_matrices[backend]['fp'] = 0 

In [24]:
for backend in backends:
    for image in images:
        if image.split(".")[0].startswith("noise"):
            if image.split(".")[0].split("_")[1] == 'f':
                if faces_detected[backend][image] == 1:
                    confusion_matrices[backend]['tp'] += 1
                else:
                    confusion_matrices[backend]['fn'] += 1
            else:
                if faces_detected[backend][image] == 1:
                    confusion_matrices[backend]['fp'] += 1
                else:
                    confusion_matrices[backend]['tn'] += 1
        else:
            if faces_detected[backend][image] == 1:
                confusion_matrices[backend]['tp'] += 1
            else:
                confusion_matrices[backend]['fn'] += 1
cm_df = pd.DataFrame.from_dict(confusion_matrices, orient='index')   
cm_df

,tp,fn,tn,fp
opencv,187,43,59,11
ssd,183,47,70,0
mtcnn,227,3,53,17
fastmtcnn,227,3,46,24
yolov8,230,0,45,25
yunet,204,26,70,0
centerface,202,28,64,6
facepp,158,72,60,10
facepp_modified,218,12,58,12


In [25]:
performance = {backend: {} for backend in backends}
for backend in backends:
    performance[backend]['time'] = detect_times[backend]
    performance[backend]['accuracy'] = (confusion_matrices[backend]['tp'] + confusion_matrices[backend]['tn']) / 300
    performance[backend]['precision'] = confusion_matrices[backend]['tp'] / (confusion_matrices[backend]['tp'] + confusion_matrices[backend]['fp'])
    performance[backend]['recall'] = confusion_matrices[backend]['tp'] / (confusion_matrices[backend]['tp'] + confusion_matrices[backend]['fn'])
    performance[backend]['f1_score'] = 2*performance[backend]['precision']*performance[backend]['recall']/(performance[backend]['precision']+performance[backend]['recall'])
scores_df = pd.DataFrame.from_dict(performance, orient='index')
scores_df

,time,accuracy,precision,recall,f1_score
opencv,53.136927,0.820000,0.944444,0.813043,0.873832
ssd,26.392200,0.843333,1.000000,0.795652,0.886199
mtcnn,574.037052,0.933333,0.930328,0.986957,0.957806
fastmtcnn,143.181745,0.910000,0.904382,0.986957,0.943867
yolov8,81.953506,0.916667,0.901961,1.000000,0.948454
yunet,21.978080,0.913333,1.000000,0.886957,0.940092
centerface,283.702315,0.886667,0.971154,0.878261,0.922374
facepp,216.413331,0.726667,0.940476,0.686957,0.793970
facepp_modified,214.459737,0.920000,0.947826,0.947826,0.947826


In [26]:
with open('detection_confusion_matrix', 'wb') as f:
    f.write(pickle.dumps(cm_df))
with open('detection_scores', 'wb') as f:
    f.write(pickle.dumps(scores_df))